In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
LOCAL_LLM = 'llama3'
CLOUD_LLM = 'llama3-70b-8192'
VECTOR_DB_URL = 'http://localhost:6333'
EMBEDDING_MODEL = 'NeuML/pubmedbert-base-embeddings'

In [ ]:
from langchain_community.vectorstores.qdrant import Qdrant
from qdrant_client import QdrantClient
from langchain_community.embeddings import sentence_transformer

def get_vector_embeddings(embedding_model: str):
    embeddings = sentence_transformer.SentenceTransformerEmbeddings(
        model_name=embedding_model
    )
    return embeddings

print("Loading vector embeddings and creating Qdrant client...")
embeddings = get_vector_embeddings(EMBEDDING_MODEL)
client = QdrantClient(VECTOR_DB_URL)
db = Qdrant(
    client=client,
    embeddings=embeddings,
    collection_name="test-collection",
)

In [ ]:
### Retrieval Grader

from langchain_core.output_parsers.json import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

# Create pydantic object for the grader result
class GraderResult(BaseModel):
    score: str = Field(
        "", 
        description="A binary score 'yes' or 'no' to indicate whether the document is relevant to the question."
    )

parser = PydanticOutputParser(pydantic_object=GraderResult)

RETRIEVAL_GRADER_PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
You are a grader assessing relevance of a retrieved document to a user question. 
If the document contains keywords related to the user question,
grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals.

{format_instructions}<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Here is the retrieved document:

{document}

Here is the user question: {question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

llm = ChatGroq(model=CLOUD_LLM)
# llm = ChatOllama(model=LOCAL_LLM, format="json", temperature=0)

prompt = PromptTemplate(
    template=RETRIEVAL_GRADER_PROMPT, 
    input_variables=["document", "question"], 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

retrieval_grader = prompt | llm | JsonOutputParser()
retrieval_grader = retrieval_grader.with_retry()

query = "When did sir Standford Raffles founded singapore?"
docs = db.similarity_search_with_score(query=query, k=5)

# Add the query to the context
for doc, score in docs:
    result = retrieval_grader.invoke({
        "document": doc,
        "question": query
    })
    print(prompt.format(document=doc, question=query))
    print(result, sep="\n\n", end="\n\n\n" + "-" * 50 + "\n\n\n")

In [3]:
### Generation Agent

from langchain_core.output_parsers.string import StrOutputParser

PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a expert in physiotherapy, you will be presented with a set of subjective and object assessment and your job is to come up with a well informed and researched differential diagnosis of the medical scenerio.
Here are examples of the subjective and objective assessments and the expected differential diagnosis:

Subjective Assessment:
The patient, Mr. Smith, is a 45-year-old male who presents to the clinic with complaints of lower back pain that has been bothering him for the past two weeks. He describes the pain as dull and achy, located in the lumbar region, with occasional radiation down his left leg. He notes that the pain worsens with prolonged sitting or standing and is relieved by lying down. He denies any recent trauma or injury but mentions that he has a history of occasional low back pain, especially after heavy lifting or prolonged periods of inactivity. He rates the pain as a 6 out of 10 on the pain scale.On physical examination, Mr. Smith appears uncomfortable but is able to walk into the examination room without assistance.

Objective Assessment:
Vital signs are within normal limits. Inspection of the lumbar spine reveals no obvious deformities or asymmetry. Palpation elicits tenderness over the paraspinal muscles of the lumbar spine, particularly on the left side. Range of motion of the lumbar spine is mildly restricted, with pain on forward flexion and left lateral bending. Straight leg raise test is positive on the left side at 45 degrees, reproducing his symptoms of radiating pain down the left leg. Neurological examination reveals intact sensation and strength in the lower extremities, with no signs of motor weakness or sensory deficits. On physical examination, Mr. Smith appears uncomfortable but is able to walk into the examination room without assistance. Vital signs are within normal limits. Inspection of the lumbar spine reveals no obvious deformities or asymmetry. Palpation elicits tenderness over the paraspinal muscles of the lumbar spine, particularly on the left side. Range of motion of the lumbar spine is mildly restricted, with pain on forward flexion and left lateral bending. Straight leg raise test is positive on the left side at 45 degrees, reproducing his symptoms of radiating pain down the left leg. Neurological examination reveals intact sensation and strength in the lower extremities, with no signs of motor weakness or sensory deficits.

Differential Diagnosis:
Based on the assessments provided, my differential diagnosis for Mr. Smith would include:
1. ...
2. ...
...

IMPORTANT:
- If the query is not related to physiotherapy or unrelated from the retrieved context, please answer with "I am sorry, I am not able to answer this question."
- Include citations and references to support your answer. <|eot_id|>
<|start_header_id|>user<|end_header_id|> 

Answer the question based only on the following context, this context should be used as source of ground truth to answer the question:

{context}

---

Answer the question based on the above context: 
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

query = """
Given the following subjective and objective assessment, provide a well informed and researched differential diagnosis

Subjective assessment:
The patient, Mr. Smith, is a 45-year-old male who presents to the clinic with complaints of lower back pain that has been bothering him for the past two weeks. He describes the pain as dull and achy, located in the lumbar region, with occasional radiation down his left leg. He notes that the pain worsens with prolonged sitting or standing and is relieved by lying down. He denies any recent trauma or injury but mentions that he has a history of occasional low back pain, especially after heavy lifting or prolonged periods of inactivity. He rates the pain as a 6 out of 10 on the pain scale.

Objective assessment:
On physical examination, Mr. Smith appears uncomfortable but is able to walk into the examination room without assistance. Vital signs are within normal limits. Inspection of the lumbar spine reveals no obvious deformities or asymmetry. Palpation elicits tenderness over the paraspinal muscles of the lumbar spine, particularly on the left side. Range of motion of the lumbar spine is mildly restricted, with pain on forward flexion and left lateral bending. Straight leg raise test is positive on the left side at 45 degrees, reproducing his symptoms of radiating pain down the left leg. Neurological examination reveals intact sensation and strength in the lower extremities, with no signs of motor weakness or sensory deficits.
"""

prompt = PromptTemplate(template=PROMPT, input_variables=["context", "question"])
# llm = ChatOllama(model=LOCAL_LLM)
llm = ChatGroq(model=CLOUD_LLM)
rag_chain = prompt | llm | StrOutputParser()
docs = db.similarity_search_with_score(query=query, k=5)

def format_retrieved_docs(docs):
    return "\n\n---\n\n".join([f"metadata: {doc.metadata}\nscore: {score}\ncontent: {doc.page_content}" for doc, score in docs])

print(prompt.format(context=format_retrieved_docs(docs), question=query))
print("Waiting for LLM response...")
stream = rag_chain.stream({
  "context": format_retrieved_docs(docs),
  "question": query
})
generation = ""
for response in stream:
    generation += response
    print(response, flush=True, end="")



NameError: name 'ChatGroq' is not defined

In [4]:
class HallucinationGraderResult(BaseModel):
    score: str = Field(
        "", 
        description="A binary score 'yes' or 'no' to indicate whether the answer is grounded in / supported by the facts provided."
    )

parser = PydanticOutputParser(pydantic_object=HallucinationGraderResult)

PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
an answer is grounded in / supported by as set of facts. Give a binary score of 'yes' or 'no' to indicate
whether the answer is grounded in / supported by the facts provided.
{format_instructions}<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Here are the facts:

----------------------------------------
{facts}
----------------------------------------

Here is the answer: {answer}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    template=PROMPT, 
    input_variables=["facts", "answer"], 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
print(prompt.format(facts=format_retrieved_docs(docs), answer=generation))
llm = ChatGroq(model=CLOUD_LLM)
hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader = hallucination_grader.with_retry()

hallucination_result = hallucination_grader.invoke({
    "facts": format_retrieved_docs(docs),
    "answer": generation
})
print(hallucination_result)

NameError: name 'BaseModel' is not defined

In [1]:
### Answer Grader Agent

class AnswerGraderResult(BaseModel):
    score: str = Field("", description="A binary score 'yes' or 'no' to indicate whether the answer is useful to resolve the question")

parser = PydanticOutputParser(pydantic_object=AnswerGraderResult)

PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
an answer is useful to resolve a question. Give a binary score of 'yes' or 'no' to indicate whether the answer is useful to resolve the question.
{format_instructions}<|eot_id|>
<|start_header_id|>user<|end_header_id|> 

Here is the answer:
----------------------------------------
{answer}
----------------------------------------

Here is the question: 
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    template=PROMPT, 
    input_variables=["answer", "question"], 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
print(prompt.format(answer=generation, question=query))
llm = ChatGroq(model=CLOUD_LLM)
answer_grader = prompt | llm | JsonOutputParser()
answer_grader = answer_grader.with_retry()

answer_result = answer_grader.invoke({
    "answer": generation,
    "question": query
})
print(answer_result)

NameError: name 'BaseModel' is not defined

In [2]:
### Router Agent

class RouterResult(BaseModel):
    datasource: str = Field(
        default="",
        description="A binary score 'yes' or 'no' to indicate whether the question should be routed to the vectorstore or web search."
    )
parser = PydanticOutputParser(pydantic_object=RouterResult)


llm = ChatGroq(model=LOCAL_LLM)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. 
    Use the vectorstore for questions on Muscle skeletal conditions, phyisotherapy domain and clinic evaluations.
    You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search.
    {format_instructions}<|eot_id|>
    <|start_header_id|>user<|end_header_id|> 
    
    Question to route: {question} <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()
question = "llm agent memory"
print(question_router.invoke({"question": question})) 
# Should return 'web_search' as the question is not related to physiotherapy or clinic evaluations

NameError: name 'local_llm' is not defined